In [93]:
from matplotlib import pyplot as plt
import numpy as np
from numpy import atleast_2d as twod
import pandas as pd
from skimage import color
import pathlib

In [94]:
# Loading data
DATA_DIR = "facial_expressions/data"
IMAGE_DIR = "facial_expressions/images"
data = pd.read_csv(f'{DATA_DIR}/legend.csv')

In [98]:
# Spliting data in to training and validation sets
X = np.array([plt.imread(IMAGE_DIR + '/' + image) for image in data['image']], dtype=object)
Y = np.array(data['emotion'].str.lower())
# Split Factor = 0.8
stop = int(0.8 * len(X))
Xtr,Xva = X[:stop],X[stop:]
Ytr,Yva = Y[:stop],Y[stop:]
# plt.imshow(color.rgb2gray(X[0]), cmap='gray')
# plt.axis('off')
# plt.show()

In [96]:
# Train the model
# Use Adaboost to find good features
# Then train the models with those features

[1]


In [99]:
# Use the train model to predict emotion on validation data set
# Calculate performance